In [2]:
import pandas as pd
import os
from bs4 import BeautifulSoup

import warnings
warnings.simplefilter(action='ignore')

In [12]:
folder = "election_results/raw"

paths = [f for f in os.listdir(folder) if f.endswith("xls")]
dfs = []

for path in paths:
    with open(f"{folder}/{path}", "r", encoding="iso-8859-9") as f:
        text = f.read()
        soup = BeautifulSoup(text, "html")
        df = pd.read_html(str(soup.findAll("table")[1]))[0]
        # df.drop(1, axis=1, inplace=True)
        city = df.iloc[2,0]
        df.drop(4, axis=1, inplace=True)
        df.drop(3, axis=1, inplace=True)
        df.drop(2, axis=1, inplace=True)
        df.drop(0, axis=1, inplace=True)

        df.columns = df.iloc[0]
        df["Seçim çevresi ve ilçe"] = df["Seçim çevresi ve ilçe"].fillna(method='ffill')
        df  = df[df["Seçim çevresi ve ilçe"] != "Seçim çevresi ve ilçe"]
        df  = df[df["Seçim çevresi ve ilçe"] != "Seçim çevresi toplamı"]

        df = df.rename(columns={"Seçim çevresi ve ilçe": "district"})
        df["city"] = city
        df = df.drop_duplicates("district", keep="first").reset_index(drop=True)
        
        dfs.append(df)
        print(city)
        
df = pd.concat(dfs, axis=0, ignore_index=True)
df

Kütahya
Gaziantep
Antalya
Van
Aydın
Ordu
Hakkari
Bayburt
Bursa
Balıkesir
Isparta
Bingöl
Bolu
Malatya
Karaman
Samsun
Gümüşhane
İstanbul
Kilis
Sakarya
Kahramanmaraş
Bitlis
Amasya
Ardahan
İzmir
Yozgat
Elazığ
Nevşehir
Sivas
Afyonkarahisar
Tokat
Adana
Denizli
Ağrı
Çanakkale
Düzce
Tunceli
Sinop
Bursa
Şanlıurfa
Konya
İzmir
Ankara
Bilecik
Kırklareli
Ankara
Siirt
Batman
Aksaray
Eskişehir
Burdur
Mardin
Çankırı
Kırşehir
Rize
Yalova
Erzincan
Niğde
İstanbul
Iğdır
İstanbul
Muş
Zonguldak
Uşak
Hatay
Artvin
Manisa
Erzurum
Mersin
Giresun
Bartın
Kocaeli
Çorum
Kars
Şırnak
Muğla
Osmaniye
Tekirdağ
Diyarbakır
Kırıkkale
Karabük
Kayseri
Kastamonu
Edirne
Trabzon
Adıyaman
Ankara


,district,Sandık sayısı,Kayıtlı seçmen sayısı,Oy kullanan seçmen sayısı,Geçerli oy sayısı,AK PARTİ,MHP,HÜDA PAR,VATAN PARTİSİ,HDP,CHP,SAADET PARTİSİ,İYİ PARTİ,CUMHUR İTTİFAKI,MİLLET İTTİFAKI,BĞMZ,city
0,Altıntaş,52,12447,11019,10720,6152,1984,23,29,64,1218,143,962,133,12,0,Kütahya
1,Domaniç,53,11945,11056,10698,6073,1304,37,50,64,1344,247,1449,117,13,0,Kütahya
2,Emet,70,15888,14465,14133,8363,2383,24,29,49,1344,424,1311,179,27,0,Kütahya
3,Gediz,154,38202,35221,34276,19464,4942,83,100,127,3823,330,4949,402,56,0,Kütahya
4,Kütahya Merkez,597,181076,165013,162388,79048,37574,252,253,1484,23636,3032,14748,2017,344,0,Kütahya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,Beypazarı,157,36755,32677,31979,14822,6814,58,87,573,3517,381,5238,434,48,7,Ankara
978,Nallıhan,121,22582,20128,19516,8890,2995,51,56,127,3661,171,3247,248,60,10,Ankara
979,Yenimahalle,1320,483390,436824,430886,147343,49092,330,953,28961,131397,7376,59713,3968,1551,202,Ankara
980,Sincan,1029,361722,327145,321802,167607,54765,521,398,12341,42449,4565,34055,4349,674,78,Ankara


In [13]:
for col in df.columns:
 if col not in ["district", "city"]:
  df[col] = pd.to_numeric(df[col])

In [14]:
df.to_csv("election_results/data.csv", index=False)